# Chapter 6 - Example: Ocean Data from NASA JPL PO.DAAC 
###  Ocean area with temperature above a given threshold

In this chapter we exemplify the use of Sea Surface Temperature (SST) data in the cloud.

- Read JPL PO.DAAC NetCDF files directly

This example analyze a time series from an area of the ocean or a point. If an area, averages SST values. Then it analyze the time series to assess when SST is above a given threshold. This could be used to study marine heatwaves, or use a threshold relevant for a marine species of interest.

# Example: Ocean Data from NASA JPL PO.DAAC
###  Ocean area with temperature above a given threshold

Here we exemplify the use of Sea Surface Temperature (SST) data in the cloud. 

This example analyze a time series from an average area of the ocean or a point. Then it analyze the time series to assess when SST is above a given threshold. This could be used to study marine heatwaves, or use a threshold relevant for a marine species of interest.

Authors:
- [Marisol Garcia-Reyes](https://github.com/marisolgr)
- [Chelle Gentemann](https://github.com/cgentemann)

Credit:
- Funding: Interagency Implementation and Advanced Concepts Team [IMPACT](https://earthdata.nasa.gov/esds/impact) for the Earth Science Data Systems (ESDS) program


In [ ]:
# Import libraries
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt 
import hvplot.xarray
import warnings

# these libraries help reading cloud data
import fsspec 
import s3fs
import requests

warnings.simplefilter("ignore")  # filter some warning messages
xr.set_options(display_style="html",keep_attrs=True)  # display dataset nicely

In [ ]:
# Input parameters
# Select either a range of lat/lon or a point. 
latr = [19, 20] # make sure lat1 > lat2 since no test is done below to simplify the code
lonr = [-158, -157] # lon1 > lon2, range -180:180. resolution daily 1km!
# time range. data range available: 2002-06-01 to 2020-01-20. [start with a short period]
dater = ['2012-01-01','2019-12-31'] # dates on the format 'YYYY-MM-DD' as string

***
## We are going to use the Multi-Scale Ultra High Resolution (MUR) Sea Surface Temperature (SST) data set
### Stored the NASA PO.DAAC Amazon (AWS) Cloud. For more info and links to the data detail and examples, [here](https://podaac.jpl.nasa.gov/dataset/MUR-JPL-L4-GLOB-v4.1).

This dataset is stored in netCDF format, which can be very slow to access over S3 glob storage using xr.open_mfdataset because it has to collect the metadata from each file. We have created a reference file using the Kerchunk library and are accessing the entire 18-year 1-km dataset using the Zarr library. So, even though the files are netCDF, we are accessing them using the Zarr library and gaining two key advantages 
- access entire dataset in ~2 min versus 50 min for netcdf
- data reads are concurrent & faster using zarr library.

To access the PO.DAAC cloud MUR SSTs:
- input your earthdata login and password  (if you don't have a login then register [here](https://urs.earthdata.nasa.gov/))
- set up credentials
- access the Kerchunk consolidated metadata file

In [ ]:
from earthdata import Auth 
auth = Auth().login()

In [ ]:
from os.path import dirname, join

def begin_s3_direct_access():
    url = "https://archive.podaac.earthdata.nasa.gov/s3credentials"
    response = requests.get(url).json()
    return s3fs.S3FileSystem(
        key=response["accessKeyId"],
        secret=response["secretAccessKey"],
        token=response["sessionToken"],
        client_kwargs={"region_name": "us-west-2"},
    )

start_year = int(dater[0][0:4])
end_year = int(dater[1][0:4])
print(start_year, end_year)



# try to break it down month by month - stops after 24 months

In [ ]:
%%time
fs = begin_s3_direct_access()
ds_sst=[]
for lyr in range(start_year,end_year):
    for imon in range(1,13):
        fstr = str(lyr)+str(imon).zfill(2)+'*.nc'
        files = fs.glob(join("podaac-ops-cumulus-protected/", "MUR-JPL-L4-GLOB-v4.1", fstr))
        # Iterate through remote_files to create a fileset
        fileset = [fs.open(file) for file in files]
        tem = xr.open_mfdataset(fileset,combine="by_coords")
        ds_sst.append(tem)
        print(lyr,imon,len(ds_sst))
ds_sst = xr.concat(ds_sst,dim='time')

# try to do year by year, stops after 2 years

In [ ]:
%%time
fs = begin_s3_direct_access()
ds_sst=[]
for lyr in range(start_year,end_year):
    fstr = str(lyr)+'*.nc'
    files = fs.glob(join("podaac-ops-cumulus-protected/", "MUR-JPL-L4-GLOB-v4.1", fstr))
    tem = xr.open_mfdataset(
        paths=[fs.open(f) for f in files],
        combine="by_coords",
    )
    tem.close()
    ds_sst.append(tem)
    print(lyr,len(ds_sst))
ds_sst = xr.concat(ds_sst,dim='time')


# Runs but after 12 hours still running

In [ ]:
# try step_1 way
#%%time
fs = begin_s3_direct_access()
flist = []
for lyr in range(start_year,end_year+1):
    for imon in range(1,13):
        fstr = str(lyr)+str(imon).zfill(2)+'*.nc'
        files = fs.glob(join("podaac-ops-cumulus-protected/", "MUR-JPL-L4-GLOB-v4.1", fstr))
        for file in files:
            flist.append(file)
print('total number of individual netcdf files:',len(flist))

In [ ]:
%%time
urls = ["s3://" + f for f in flist]
so = dict(mode='rb', anon=True, default_fill_cache=False, default_cache_type='first')

In [ ]:
def get_data(u,ds_sst):
    with fs.open(u, **so) as infile:
        tem = xr.open_dataset(infile)
    ds_sst.append(tem)

In [ ]:
%%time
import dask
ds_sst=[]
_ = dask.compute(*[dask.delayed(get_data)(u,ds_sst) for u in urls], retries=10);
ds_sst = xr.concat(ds_sst,dim='time')
ds_sst

## Now that we know what the file contains, we select our data (region and time), operate on it if needed (if a region, average), and download only that data 
It takes a while, given the high resolution of the data. So, be patient.... and if you're only testing, might want to choose a small region and a short time period first. 

In [ ]:
lyr

In [ ]:
sst = ds_sst['analysed_sst'].sel(time = slice(dater[0],dater[1]),
                                        lat  = slice(latr[0], latr[1]), 
                                        lon  = slice(lonr[0], lonr[1])
                                       ).mean(dim={'lat','lon'}, 
                                              skipna=True) # skip 'not a number' values and keep attributes
sst = sst-273.15 # transform Kelving to degrees Celsius
sst.attrs['units']='deg C' #update units in metadata
sst

***
## Let's plot the data using two different libraries.
#### - `matplotlib` that we already learn. It makes static and very nice figures that can be customized.

In [ ]:
sst.plot() 

#### - `hovplot` is a more interactive library for web display. it provides you with the data details when you hover your cursor over the figure. Very nice for inspecting the data.

In [ ]:
sst.hvplot(grid=True)

***
## Now, let's analyze our data.
#### First, the a basic climatology and anomalies, and plotting using `hovplot`.

In [ ]:
# Calculate the climatology
sst_climatology = sst.groupby('time.dayofyear').mean('time',skipna=False) # Group by day, all years. skipna ignore missing values (NaN=Not a Number)
sst_climstd = sst.groupby('time.dayofyear').std('time',skipna=False) # Calculate standard deviation. 

ds = xr.Dataset({'clim':sst_climatology,'+Std':sst_climatology+sst_climstd,'-Std':sst_climatology-sst_climstd})
ds.hvplot(color=['k','grey','grey'], grid=True, title='SST Climatology')

In [ ]:
# Calculate the anomalies
sst_anomaly = sst.groupby('time.dayofyear')-sst_climatology 
sst_anomaly_monthly = sst_anomaly.resample(time='1MS', loffset='15D').mean(skipna=False) # calculate monthly anomalies/smoothing

sst_anomaly.hvplot.area(x='time',grid=True, title='SST Anomalies')

***
## We analyze the data further by choosing a threshold.

- One way is to set a threshold that has some relevance.  For example, a thermal threshold for a marine species we are studying. 

- Another way is choosing the maximum value in the climatology (mean value + 1 standard deviation), which we can calculate or read by hovering our cursor over the climatology plot above.

### Once the threshold is choosen, we identify when SST is over that threshold, and count how many days that occurred each year

In [ ]:
# first, we define a function that take a threshold value, and analyze and plot our data
def SST_above(thr):
    
    fig, axs = plt.subplots(1,2,figsize=(16,4))

    # first part - values above threshold
    sst.plot(ax=axs[0],linewidth=1)
    sst.where(sst>=thr).plot.line('r.',markersize=3,ax=axs[0]) # test when data is equal or greater than the threshold. a is a logical array (True/False values)
    axs[0].grid(True, alpha=0.3)

    # second part - days per year above threshold
    hot = sst.where(sst>=thr,drop=True)
    xr.plot.hist(hot.time.dt.year,ax=axs[1],bins=np.arange(2011.5,2020.5),rwidth=.8)
    plt.ylabel('No. days above '+str(np.round(thr,1))+'C')
    plt.grid(True, alpha=0.3)
    plt.show() # display

## Second, the actual analysis: two examples ##
### Maximum climatology threshold
thr = ds['+Std'].max().data # setting threshold as maximum climatological value: mean + 1 standard deviation
print('Max climatological SST = ',np.round(thr,1),'C')
SST_above(thr) # Call function we defined

### A relevant threshold. 
# For example, for hawaii (the select region), 28C is a relevant threshold for coral bleaching (https://coralreefwatch.noaa.gov/product/5km/tutorial/crw08a_bleaching_threshold.php)
thr = 28
print('\n\nBiologically relevant SST = ',thr,'C')
SST_above(thr) # Call function

***
#### Now, a different analsys of anomalously warm SST days. 
## Marine heatwaves
Defined as any period with SST anomalies above the threshold set by the 90th percentile value of a given period SST anomalies - in this case our data time period.

In [ ]:
# First, calculate the threshold: 90th percentile
thr = np.percentile(sst_anomaly, 90)

fig, axs = plt.subplots(3,1,figsize=(16,16))

# Same plot as in our function above, but this time we are plotting the anomalies.
sst_anomaly.plot(ax=axs[0],linewidth=1)
sst_anomaly.where(sst_anomaly>=thr).plot.line('r.',markersize=3,ax=axs[0]) # test when data is equal or greater than the threshold. a is a logical array (True/False values)
axs[0].grid(True, alpha=0.3)
axs[0].title.set_text('Location: '+str(latr)+'$^\circ$N, '+str(lonr)+'$^\circ$W')

# Now plot on the original data (not anomalies)
sst.plot(ax=axs[1],linewidth=1)
sst.where(sst_anomaly>=thr).plot.line('r.',markersize=3,ax=axs[1]) # test when data is equal or greater than the threshold. a is a logical array (True/False values)
axs[1].grid(True, alpha=0.3)

# Plot of marine heatwave days  per year
hot = sst_anomaly.where(sst_anomaly>=thr,drop=True)
xr.plot.hist(hot.time.dt.year,ax=axs[2],bins=np.arange(2011.5,2020.5),rwidth=.8)
plt.ylabel('No. days SSTa > '+str(np.round(thr,1))+'C')
plt.grid(True, alpha=0.3)
plt.show()

# print the numbers of days
hot.groupby('time.year').count().data 

***
## Finally, let's explore the SST field around our selected point or region durring the warmest day or coldest day.

In [ ]:
# Find out max and min SST values and the date when they occur
minv = sst.min() # find mininum SST value
mindate = sst[sst==minv].time.data # find when this min value occurred
maxv = sst.max() # find maximum SST value
maxdate = sst[sst==maxv].time.data # find when the max value occurred

In [ ]:
# define a function that go back to the SST data in the cloud, but we now load a different subset 
# an specific day, but now always a region: the region selected or a region around the selected point
def select_area(day):   # the function input is a day
    if len(latr)==1: # if input data was one point
        sst2 = ds_sst.sel(time = day, 
                            lat  = slice(latr-2,latr+2),
                            lon  = slice(lonr-2,lonr+2)
                            )
    else: # if input data was a region
        sst2 = ds_sst.sel(time = day,
                            lat  = slice(latr[0], latr[1]), 
                            lon  = slice(lonr[0], lonr[1])
                            )
    sst3 = sst2['analysed_sst']-273.15
    mask = sst2['mask'].where(sst2['mask']<2)
    sst3 = sst3*mask
    return sst3 # returns the data array of the region at the given date (the region is the defined at the beginning of the script)

In [ ]:
# plot warmest day
msst = select_area(maxdate) # call function with day of warmest SST
msst.hvplot.quadmesh(x='lon',y='lat',coastline=True, clabel='T [C]', cmap='coolwarm', title=str(maxdate[0])[:10])
# this image plot also gives you extra information when you hover your cursor around

In [ ]:
# plot coolest day
msst = select_area(mindate) # call function with day of coolest SST
msst.hvplot.quadmesh(x='lon',y='lat',coastline=True, clabel='T [C]', cmap='coolwarm', title=str(mindate[0])[:10])

***
# Resources

For the cloud and data in the cloud, see resources listed in Chapter 5.

Resources specifically for this chapter:
- [MUR SST Data](https://registry.opendata.aws/mur/). SST data in the cloud, with references the official datta website, examples and other resources.
- [Pangeo OSM2020 Tutorial](https://github.com/pangeo-gallery/osm2020tutorial). This is a very good tutorial for ocean application and cloud computing. Plenty of examples. Many of the commands here are from this tutorial.

### About MHW

- [Marine heatwaves](http://www.marineheatwaves.org/all-about-mhws.html). A good place to begin to get info about the subject.
- [Marine heatwaves code](https://github.com/ecjoliver/marineHeatWaves). Marine heatwaves code from E. Oliver. 

### If you want to learn more:
- [Methods for accessing a AWS bucket](https://docs.aws.amazon.com/AmazonS3/latest/userguide/access-bucket-intro.html). Bucket is the name of the cloud storage object. S3 stands for Amazon's Simple Storage Service.
- [hvplot site](https://hvplot.holoviz.org/index.html). Plotting tool used here.
- [zarr](https://zarr.readthedocs.io/en/stable/) Big data storage formatt
